# Start with loading the glove embedding

In [1]:
import numpy as np 
%matplotlib inline

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
from torch import nn
import math
from functools import partial
from pathlib import Path
from tqdm import tqdm
#import rich
from typing import List, Tuple, Optional, Dict, Any
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#import transformers
#import tokenizers
#import datasets
#import zipfile
#from huggingface_hub import hf_hub_download
device = 'cpu'

In [18]:
target_file = "glove.6B.50d.txt"
vocabulary = []
vectors = []
emb_dic = {}
with open(target_file, "r", encoding="utf8") as f:
    for l in tqdm(f.readlines() ):
        word, *vector = l.split()
        vocabulary.append(word)
        vector_t = torch.tensor([float(v) for v in vector])
        vectors.append(vector_t)
        emb_dic[word] = vector_t
vectors = torch.stack(vectors)

100%|██████████| 400001/400001 [00:08<00:00, 49579.60it/s]


In [92]:
# somtehing from week 4
class NewsEncoder(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.activation_fn = nn.ReLU

        #self.l_1 = nn.Linear(in_features=int(features_cat_size),
        #                  out_features=l_1_hidden_size)
        

    def forward(self, x): # x is a string of words - title
        words = x.split(" ")
        vectors = []
        for word in words:
            print(word)
            vectors.append(torch.Tensor(emb_dic[word.lower()]))
        #features_final = self.l_1(x)
        e_s = torch.stack(vectors)
        scaled_dot = nn.functional.scaled_dot_product_attention(e_s,e_s,e_s)
        return scaled_dot

In [93]:
model = NewsEncoder(0) # TODO - I need to change this with a number that will correspond to 

In [94]:
test1 = "Breaking news about this"
test2 = "This has happened again"

In [95]:
model.forward(test1)

Breaking
news
about
this


tensor([[ 1.0157e-02,  2.4989e-01,  4.3846e-02,  1.5625e-01,  2.2384e-01,
          6.2950e-02, -3.3575e-01, -1.8143e-02, -1.0717e-01,  4.8769e-03,
         -4.0176e-01, -3.0917e-01, -6.1963e-01,  6.2024e-02,  8.9128e-01,
         -9.4411e-03, -2.2247e-01, -2.6615e-01, -4.3537e-01, -1.8817e-01,
          2.5553e-01,  4.0876e-01,  3.6024e-01, -5.7414e-02,  4.3141e-01,
         -1.6105e+00, -1.9780e-01,  2.5115e-01,  4.7579e-01, -4.5814e-03,
          3.3320e+00,  2.1369e-02,  5.6244e-02, -1.5367e-01, -5.2320e-01,
         -1.7383e-01, -3.3079e-02, -6.0352e-02,  6.2080e-04, -6.7409e-02,
         -8.9417e-02,  1.3494e-01,  2.8677e-02,  2.5934e-01,  4.0715e-03,
          1.2172e-01,  3.7067e-03,  2.5722e-01, -2.0290e-01, -2.9370e-02],
        [-3.2087e-02,  4.4404e-01,  4.0762e-01,  7.8165e-01,  3.1579e-02,
         -8.1479e-01, -1.0370e+00, -1.5414e-01, -1.2605e-01, -5.7964e-01,
         -4.1474e-01, -2.5222e-01, -3.8301e-01,  2.3935e-01,  1.0095e+00,
          1.1953e-01, -1.0602e+00, -1

In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)#, weight_decay=0.3)

NameError: name 'optim' is not defined

In [ ]:
# Test the forward pass with dummy data
out = model([])
print("Output shape:", out.size())
print(f"Output logits:\n{out.detach().cpu().numpy()}")
print(f"Output probabilities:\n{out.softmax(1).detach().cpu().numpy()}")